# Mathematical Foundations: Linear Algebra and Network Theory in Systems Biology

**BIO559R - Introduction to Systems Biology**  
**Module 2: Mathematical Foundations**

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Apply linear algebra** to analyze metabolic networks and pathways
2. **Perform flux balance analysis** using stoichiometric matrices
3. **Use Principal Component Analysis** for dimensionality reduction in omics data
4. **Analyze network topology** using graph theory metrics
5. **Identify network motifs** and their biological significance
6. **Visualize complex networks** using Python and R
7. **Interpret biological meaning** from mathematical network analysis

## Prerequisites

- Basic linear algebra (matrices, vectors, eigenvalues)
- Python programming fundamentals
- Understanding of biological networks

---

## Setup and Imports

In [ ]:
# Import Python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.sparse import csr_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import networkx as nx
import seaborn as sns

# Load R magic for visualization
%load_ext rpy2.ipython

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries loaded successfully!")
print(f"NumPy version: {np.__version__}")
print(f"NetworkX version: {nx.__version__}")

## Part 1: Linear Algebra Fundamentals for Biology

### Matrix Operations in Biological Context

In systems biology, matrices represent:
- **Stoichiometric matrices**: Metabolic network structure
- **Adjacency matrices**: Network connectivity
- **Expression matrices**: Gene expression data
- **Interaction matrices**: Protein-protein interactions

Key operations:
- **Matrix multiplication**: Pathway flux calculations
- **Null space**: Steady-state flux distributions
- **Eigendecomposition**: Principal components, network centrality
- **Matrix inversion**: Solving linear systems

### Exercise 1.1: Basic Matrix Operations

In [ ]:
# Create example biological matrices

# Stoichiometric matrix for a simple pathway: A -> B -> C
# Reactions: R1: A -> B, R2: B -> C, R3: A -> (external)
S = np.array([
    [-1,  0,  -1],  # Metabolite A
    [ 1, -1,   0],  # Metabolite B  
    [ 0,  1,   0]   # Metabolite C
])

print("Stoichiometric Matrix S:")
print(S)
print(f"Shape: {S.shape} (metabolites x reactions)")

# Example flux vector
v = np.array([2.0, 1.5, 0.5])  # Flux through each reaction
print(f"\nFlux vector v: {v}")

# Calculate metabolite production rates
production_rates = S @ v
print(f"\nMetabolite production rates (S·v): {production_rates}")

# Check if this is a steady state (all rates should be zero)
is_steady_state = np.allclose(production_rates, 0, atol=1e-10)
print(f"Is steady state? {is_steady_state}")

# Find a steady-state flux distribution
# For steady state: S·v = 0
# We need to find the null space of S
U, s, Vt = linalg.svd(S)
null_space = Vt[len(s):, :].T
print(f"\nNull space of S (steady-state flux modes):")
print(null_space)

# Verify steady state
if null_space.size > 0:
    steady_flux = null_space[:, 0] * 2  # Scale the first mode
    steady_rates = S @ steady_flux
    print(f"\nSteady-state flux: {steady_flux}")
    print(f"Production rates: {steady_rates}")
    print(f"Is steady state? {np.allclose(steady_rates, 0)}")

## Part 2: Metabolic Flux Balance Analysis

### Theory

Flux Balance Analysis (FBA) uses linear programming to find optimal flux distributions in metabolic networks:

**Maximize**: $c^T v$ (objective function)

**Subject to**:
- $S \cdot v = 0$ (steady-state constraint)
- $v_{min} \leq v \leq v_{max}$ (flux bounds)

Where:
- $S$ is the stoichiometric matrix
- $v$ is the flux vector
- $c$ is the objective coefficient vector

### Exercise 2.1: Simple Metabolic Network Analysis

In [ ]:
# Create a more complex metabolic network
# Network: Glucose -> G6P -> F6P -> Pyruvate
#                     |-> Biomass

# Reactions:
# R1: Glucose -> G6P
# R2: G6P -> F6P  
# R3: F6P -> Pyruvate
# R4: G6P -> Biomass
# R5: Glucose uptake (external)

metabolites = ['Glucose', 'G6P', 'F6P', 'Pyruvate', 'Biomass']
reactions = ['R1_Glc_to_G6P', 'R2_G6P_to_F6P', 'R3_F6P_to_Pyr', 'R4_G6P_to_Bio', 'R5_Glc_uptake']

# Stoichiometric matrix
S_metabolic = np.array([
    [-1,  0,  0,  0,  1],  # Glucose
    [ 1, -1,  0, -1,  0],  # G6P
    [ 0,  1, -1,  0,  0],  # F6P
    [ 0,  0,  1,  0,  0],  # Pyruvate
    [ 0,  0,  0,  1,  0]   # Biomass
])

print("Metabolic Network Stoichiometric Matrix:")
print("Metabolites:", metabolites)
print("Reactions:", reactions)
print(S_metabolic)

# Create DataFrame for better visualization
S_df = pd.DataFrame(S_metabolic, 
                   index=metabolites, 
                   columns=reactions)
print("\nStoichiometric Matrix as DataFrame:")
print(S_df)

In [ ]:
# Analyze the network structure
def analyze_stoichiometric_matrix(S, metabolite_names, reaction_names):
    """
    Analyze properties of a stoichiometric matrix
    """
    print(f"Matrix dimensions: {S.shape[0]} metabolites × {S.shape[1]} reactions")
    print(f"Matrix rank: {linalg.matrix_rank(S)}")
    
    # Find null space (steady-state flux modes)
    U, s, Vt = linalg.svd(S)
    null_space_dim = S.shape[1] - linalg.matrix_rank(S)
    print(f"Null space dimension: {null_space_dim}")
    
    if null_space_dim > 0:
        null_vectors = Vt[-null_space_dim:, :].T
        print("\nSteady-state flux modes:")
        for i in range(null_space_dim):
            mode = null_vectors[:, i]
            print(f"Mode {i+1}:")
            for j, (reaction, flux) in enumerate(zip(reaction_names, mode)):
                if abs(flux) > 1e-10:
                    print(f"  {reaction}: {flux:.3f}")
        
        return null_vectors
    else:
        print("No steady-state flux modes found (overdetermined system)")
        return None

flux_modes = analyze_stoichiometric_matrix(S_metabolic, metabolites, reactions)

### Exercise 2.2: Flux Balance Analysis Implementation

In [ ]:
from scipy.optimize import linprog

def flux_balance_analysis(S, c, bounds=None):
    """
    Perform Flux Balance Analysis using linear programming
    
    Parameters:
    S: stoichiometric matrix
    c: objective coefficients (negative for maximization)
    bounds: flux bounds for each reaction
    """
    n_reactions = S.shape[1]
    
    # Default bounds: allow reversible reactions
    if bounds is None:
        bounds = [(-1000, 1000)] * n_reactions
    
    # Equality constraints: S·v = 0 (steady state)
    A_eq = S
    b_eq = np.zeros(S.shape[0])
    
    # Solve linear programming problem
    result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
    
    return result

# Define objective: maximize biomass production (R4)
objective = np.zeros(len(reactions))
objective[3] = -1  # Negative because linprog minimizes

# Define flux bounds
flux_bounds = [
    (0, 1000),    # R1: irreversible
    (0, 1000),    # R2: irreversible
    (0, 1000),    # R3: irreversible
    (0, 1000),    # R4: irreversible (biomass)
    (0, 10)       # R5: glucose uptake limited
]

# Perform FBA
fba_result = flux_balance_analysis(S_metabolic, objective, flux_bounds)

print("FBA Results:")
print(f"Optimization successful: {fba_result.success}")
print(f"Optimal objective value: {-fba_result.fun:.3f}")
print("\nOptimal flux distribution:")
for reaction, flux in zip(reactions, fba_result.x):
    print(f"{reaction}: {flux:.3f}")

# Verify steady state
production_rates = S_metabolic @ fba_result.x
print("\nMetabolite production rates (should be ~0):")
for metabolite, rate in zip(metabolites, production_rates):
    print(f"{metabolite}: {rate:.6f}")

In [ ]:
# Create visualization data for FBA results
fba_df = pd.DataFrame({
    'reaction': reactions,
    'flux': fba_result.x,
    'reaction_type': ['Central_Metabolism', 'Central_Metabolism', 'Central_Metabolism', 'Biomass', 'Uptake']
})

print("FBA results prepared for visualization:")
print(fba_df)

In [ ]:
%%R -i fba_df -w 12 -h 8

library(ggplot2)
library(dplyr)

# Create FBA results visualization
p1 <- ggplot(fba_df, aes(x = reorder(reaction, flux), y = flux, fill = reaction_type)) +
    geom_col(alpha = 0.8) +
    coord_flip() +
    scale_fill_manual(values = c(
        "Central_Metabolism" = "#2ca02c",
        "Biomass" = "#d62728",
        "Uptake" = "#1f77b4"
    )) +
    labs(
        title = "Flux Balance Analysis Results",
        subtitle = "Optimal flux distribution for biomass maximization",
        x = "Reaction",
        y = "Flux (mmol/gDW/h)",
        fill = "Reaction Type"
    ) +
    theme_classic() +
    theme(
        plot.title = element_text(size = 16, hjust = 0.5),
        plot.subtitle = element_text(size = 12, hjust = 0.5),
        axis.title = element_text(size = 12),
        axis.text = element_text(size = 10),
        legend.title = element_text(size = 12),
        legend.text = element_text(size = 10)
    )

print(p1)

## Part 3: Principal Component Analysis for Omics Data

### Theory

Principal Component Analysis (PCA) reduces dimensionality while preserving variance:

1. **Standardize** the data matrix $X$
2. **Compute** covariance matrix $C = \frac{1}{n-1}X^TX$
3. **Find** eigenvalues and eigenvectors of $C$
4. **Project** data onto principal components

Applications:
- Gene expression analysis
- Metabolomics data exploration
- Quality control in omics experiments

### Exercise 3.1: PCA on Simulated Gene Expression Data

In [ ]:
# Generate simulated gene expression data
def generate_expression_data(n_samples=100, n_genes=1000, n_conditions=3):
    """
    Generate simulated gene expression data with different conditions
    """
    np.random.seed(42)
    
    # Base expression levels
    base_expression = np.random.lognormal(mean=2, sigma=1, size=n_genes)
    
    data = []
    labels = []
    
    for condition in range(n_conditions):
        n_samples_condition = n_samples // n_conditions
        
        for sample in range(n_samples_condition):
            # Add condition-specific effects
            condition_effect = np.random.normal(0, 0.5, n_genes)
            if condition == 1:  # Condition 1: upregulate first 100 genes
                condition_effect[:100] += 2
            elif condition == 2:  # Condition 2: downregulate genes 100-200
                condition_effect[100:200] -= 1.5
            
            # Add noise
            noise = np.random.normal(0, 0.3, n_genes)
            
            expression = base_expression * np.exp(condition_effect + noise)
            data.append(expression)
            labels.append(f'Condition_{condition}')
    
    return np.array(data), labels

# Generate data
expression_data, condition_labels = generate_expression_data()
print(f"Generated expression data: {expression_data.shape}")
print(f"Conditions: {set(condition_labels)}")

# Log-transform and standardize
log_expression = np.log2(expression_data + 1)
scaler = StandardScaler()
scaled_expression = scaler.fit_transform(log_expression)

print(f"Data after log-transformation and scaling: {scaled_expression.shape}")
print(f"Mean: {scaled_expression.mean():.3f}, Std: {scaled_expression.std():.3f}")

In [ ]:
# Perform PCA
pca = PCA(n_components=10)
pca_result = pca.fit_transform(scaled_expression)

print("PCA Results:")
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Cumulative explained variance: {np.cumsum(pca.explained_variance_ratio_)}")

# Create PCA results DataFrame
pca_df = pd.DataFrame({
    'PC1': pca_result[:, 0],
    'PC2': pca_result[:, 1],
    'PC3': pca_result[:, 2],
    'condition': condition_labels,
    'sample_id': [f'Sample_{i}' for i in range(len(condition_labels))]
})

# Variance explained data
variance_df = pd.DataFrame({
    'PC': [f'PC{i+1}' for i in range(10)],
    'variance_explained': pca.explained_variance_ratio_,
    'cumulative_variance': np.cumsum(pca.explained_variance_ratio_)
})

print("\nPCA data prepared for visualization")
print(f"PCA DataFrame shape: {pca_df.shape}")

In [ ]:
%%R -i pca_df -i variance_df -w 14 -h 10

library(ggplot2)
library(dplyr)
library(gridExtra)

# PCA scatter plot
p2a <- ggplot(pca_df, aes(x = PC1, y = PC2, color = condition)) +
    geom_point(size = 3, alpha = 0.7) +
    stat_ellipse(level = 0.68, linetype = "dashed") +
    scale_color_manual(values = c(
        "Condition_0" = "#1f77b4",
        "Condition_1" = "#ff7f0e",
        "Condition_2" = "#2ca02c"
    )) +
    labs(
        title = "PCA of Gene Expression Data",
        x = paste0("PC1 (", round(variance_df$variance_explained[1] * 100, 1), "% variance)"),
        y = paste0("PC2 (", round(variance_df$variance_explained[2] * 100, 1), "% variance)"),
        color = "Condition"
    ) +
    theme_classic() +
    theme(
        plot.title = element_text(size = 14, hjust = 0.5),
        axis.title = element_text(size = 12),
        legend.title = element_text(size = 12)
    )

# Variance explained plot
p2b <- ggplot(variance_df[1:5,], aes(x = PC, y = variance_explained)) +
    geom_col(fill = "steelblue", alpha = 0.7) +
    geom_text(aes(label = paste0(round(variance_explained * 100, 1), "%")), 
              vjust = -0.5, size = 3) +
    labs(
        title = "Variance Explained by Principal Components",
        x = "Principal Component",
        y = "Proportion of Variance Explained"
    ) +
    theme_classic() +
    theme(
        plot.title = element_text(size = 14, hjust = 0.5),
        axis.title = element_text(size = 12)
    )

grid.arrange(p2a, p2b, nrow = 2, heights = c(2, 1))

### Exercise 3.2: Gene Loading Analysis

In [ ]:
# Analyze gene loadings (contributions to PCs)
def analyze_gene_loadings(pca_model, n_top_genes=20):
    """
    Analyze which genes contribute most to each principal component
    """
    loadings = pca_model.components_
    
    loading_results = []
    
    for pc in range(min(3, loadings.shape[0])):  # Analyze first 3 PCs
        pc_loadings = loadings[pc, :]
        
        # Get top positive and negative loadings
        top_positive_idx = np.argsort(pc_loadings)[-n_top_genes//2:]
        top_negative_idx = np.argsort(pc_loadings)[:n_top_genes//2]
        
        for idx in top_positive_idx:
            loading_results.append({
                'PC': f'PC{pc+1}',
                'gene': f'Gene_{idx}',
                'loading': pc_loadings[idx],
                'direction': 'Positive'
            })
        
        for idx in top_negative_idx:
            loading_results.append({
                'PC': f'PC{pc+1}',
                'gene': f'Gene_{idx}',
                'loading': pc_loadings[idx],
                'direction': 'Negative'
            })
    
    return pd.DataFrame(loading_results)

# Analyze loadings
loadings_df = analyze_gene_loadings(pca)
print("Gene loadings analysis:")
print(loadings_df.head(10))

# Summary statistics
print("\nTop contributing genes by PC:")
for pc in ['PC1', 'PC2', 'PC3']:
    pc_data = loadings_df[loadings_df['PC'] == pc]
    top_gene = pc_data.loc[pc_data['loading'].abs().idxmax()]
    print(f"{pc}: {top_gene['gene']} (loading: {top_gene['loading']:.3f})")

In [ ]:
%%R -i loadings_df -w 12 -h 8

library(ggplot2)
library(dplyr)

# Create gene loadings plot
p3 <- loadings_df %>%
    filter(PC %in% c("PC1", "PC2")) %>%
    ggplot(aes(x = reorder(gene, abs(loading)), y = loading, fill = direction)) +
    geom_col() +
    facet_wrap(~PC, scales = "free_y") +
    coord_flip() +
    scale_fill_manual(values = c("Positive" = "#d62728", "Negative" = "#1f77b4")) +
    labs(
        title = "Gene Loadings on Principal Components",
        subtitle = "Top contributing genes to PC1 and PC2",
        x = "Gene",
        y = "Loading Value",
        fill = "Direction"
    ) +
    theme_classic() +
    theme(
        plot.title = element_text(size = 16, hjust = 0.5),
        plot.subtitle = element_text(size = 12, hjust = 0.5),
        axis.title = element_text(size = 12),
        axis.text.y = element_text(size = 8),
        strip.text = element_text(size = 12)
    )

print(p3)

## Part 4: Network Theory and Graph Analysis

### Theory

Biological networks can be represented as graphs:
- **Nodes**: Biological entities (genes, proteins, metabolites)
- **Edges**: Interactions or relationships

Key network metrics:
- **Degree**: Number of connections per node
- **Centrality**: Importance of nodes in the network
- **Clustering**: Local connectivity patterns
- **Path length**: Distance between nodes

### Exercise 4.1: Protein-Protein Interaction Network

In [ ]:
# Create a simulated protein-protein interaction network
def create_ppi_network(n_proteins=50, connection_prob=0.1):
    """
    Create a simulated protein-protein interaction network
    """
    # Create random network
    G = nx.erdos_renyi_graph(n_proteins, connection_prob)
    
    # Add protein names
    protein_names = [f'Protein_{i:02d}' for i in range(n_proteins)]
    mapping = {i: protein_names[i] for i in range(n_proteins)}
    G = nx.relabel_nodes(G, mapping)
    
    # Add some hub proteins (highly connected)
    hub_proteins = np.random.choice(protein_names, 5, replace=False)
    for hub in hub_proteins:
        # Connect hub to additional random proteins
        targets = np.random.choice(protein_names, 8, replace=False)
        for target in targets:
            if hub != target:
                G.add_edge(hub, target)
    
    return G, hub_proteins

# Create PPI network
ppi_network, hub_proteins = create_ppi_network()

print(f"PPI Network created:")
print(f"Nodes: {ppi_network.number_of_nodes()}")
print(f"Edges: {ppi_network.number_of_edges()}")
print(f"Hub proteins: {hub_proteins}")

# Calculate network metrics
def calculate_network_metrics(G):
    """
    Calculate various network topology metrics
    """
    metrics = {}
    
    # Basic metrics
    metrics['degree'] = dict(G.degree())
    metrics['clustering'] = nx.clustering(G)
    
    # Centrality measures
    metrics['betweenness'] = nx.betweenness_centrality(G)
    metrics['closeness'] = nx.closeness_centrality(G)
    metrics['eigenvector'] = nx.eigenvector_centrality(G, max_iter=1000)
    
    return metrics

network_metrics = calculate_network_metrics(ppi_network)
print("\nNetwork metrics calculated successfully!")

In [ ]:
# Create network analysis DataFrame
network_df = pd.DataFrame({
    'protein': list(ppi_network.nodes()),
    'degree': [network_metrics['degree'][node] for node in ppi_network.nodes()],
    'clustering': [network_metrics['clustering'][node] for node in ppi_network.nodes()],
    'betweenness': [network_metrics['betweenness'][node] for node in ppi_network.nodes()],
    'closeness': [network_metrics['closeness'][node] for node in ppi_network.nodes()],
    'eigenvector': [network_metrics['eigenvector'][node] for node in ppi_network.nodes()],
    'is_hub': [protein in hub_proteins for protein in ppi_network.nodes()]
})

print("Network analysis DataFrame:")
print(network_df.head())
print(f"\nTop 5 proteins by degree:")
print(network_df.nlargest(5, 'degree')[['protein', 'degree', 'is_hub']])

In [ ]:
%%R -i network_df -w 14 -h 10

library(ggplot2)
library(dplyr)
library(gridExtra)

# Degree distribution
p4a <- ggplot(network_df, aes(x = degree, fill = is_hub)) +
    geom_histogram(bins = 15, alpha = 0.7) +
    scale_fill_manual(values = c("FALSE" = "lightblue", "TRUE" = "red")) +
    labs(
        title = "Protein Degree Distribution",
        x = "Degree (Number of Interactions)",
        y = "Count",
        fill = "Hub Protein"
    ) +
    theme_classic()

# Centrality comparison
p4b <- ggplot(network_df, aes(x = degree, y = betweenness, color = is_hub, size = eigenvector)) +
    geom_point(alpha = 0.7) +
    scale_color_manual(values = c("FALSE" = "blue", "TRUE" = "red")) +
    scale_size_continuous(range = c(1, 4)) +
    labs(
        title = "Network Centrality Analysis",
        x = "Degree Centrality",
        y = "Betweenness Centrality",
        color = "Hub Protein",
        size = "Eigenvector\nCentrality"
    ) +
    theme_classic()

# Clustering coefficient vs degree
p4c <- ggplot(network_df, aes(x = degree, y = clustering, color = is_hub)) +
    geom_point(size = 2, alpha = 0.7) +
    geom_smooth(method = "lm", se = FALSE, color = "black", linetype = "dashed") +
    scale_color_manual(values = c("FALSE" = "blue", "TRUE" = "red")) +
    labs(
        title = "Clustering vs Degree",
        x = "Degree",
        y = "Clustering Coefficient",
        color = "Hub Protein"
    ) +
    theme_classic()

grid.arrange(p4a, p4b, p4c, nrow = 2, ncol = 2, heights = c(1, 1))

### Exercise 4.2: Network Motif Analysis

In [ ]:
# Create a directed gene regulatory network
def create_gene_regulatory_network(n_genes=20):
    """
    Create a simulated gene regulatory network with common motifs
    """
    G = nx.DiGraph()
    
    # Add genes
    genes = [f'Gene_{i:02d}' for i in range(n_genes)]
    G.add_nodes_from(genes)
    
    # Add some common network motifs
    
    # 1. Feed-forward loops
    G.add_edges_from([
        ('Gene_00', 'Gene_01'),  # X -> Y
        ('Gene_00', 'Gene_02'),  # X -> Z
        ('Gene_01', 'Gene_02'),  # Y -> Z
    ])
    
    # 2. Feedback loops
    G.add_edges_from([
        ('Gene_03', 'Gene_04'),  # A -> B
        ('Gene_04', 'Gene_03'),  # B -> A (mutual regulation)
    ])
    
    # 3. Fan-out (one regulator, multiple targets)
    master_regulator = 'Gene_05'
    targets = ['Gene_06', 'Gene_07', 'Gene_08', 'Gene_09']
    for target in targets:
        G.add_edge(master_regulator, target)
    
    # 4. Fan-in (multiple regulators, one target)
    regulators = ['Gene_10', 'Gene_11', 'Gene_12']
    target = 'Gene_13'
    for regulator in regulators:
        G.add_edge(regulator, target)
    
    # Add some random edges
    remaining_genes = genes[14:]
    for i in range(len(remaining_genes) - 1):
        if np.random.random() > 0.5:
            G.add_edge(remaining_genes[i], remaining_genes[i+1])
    
    return G

# Create GRN
grn = create_gene_regulatory_network()

print(f"Gene Regulatory Network:")
print(f"Nodes: {grn.number_of_nodes()}")
print(f"Edges: {grn.number_of_edges()}")

# Analyze network motifs
def find_network_motifs(G):
    """
    Find common 3-node motifs in directed networks
    """
    motifs = {
        'feed_forward_loops': 0,
        'feedback_loops': 0,
        'mutual_regulation': 0
    }
    
    nodes = list(G.nodes())
    
    # Check all triplets of nodes
    for i in range(len(nodes)):
        for j in range(i+1, len(nodes)):
            for k in range(j+1, len(nodes)):
                triplet = [nodes[i], nodes[j], nodes[k]]
                
                # Check for feed-forward loop: A->B, A->C, B->C
                if (G.has_edge(triplet[0], triplet[1]) and 
                    G.has_edge(triplet[0], triplet[2]) and 
                    G.has_edge(triplet[1], triplet[2])):
                    motifs['feed_forward_loops'] += 1
    
    # Check for mutual regulation (2-node motif)
    for i in range(len(nodes)):
        for j in range(i+1, len(nodes)):
            if G.has_edge(nodes[i], nodes[j]) and G.has_edge(nodes[j], nodes[i]):
                motifs['mutual_regulation'] += 1
    
    return motifs

motifs = find_network_motifs(grn)
print(f"\nNetwork motifs found:")
for motif, count in motifs.items():
    print(f"{motif}: {count}")

In [ ]:
# Analyze GRN properties
grn_metrics = calculate_network_metrics(grn.to_undirected())  # Convert to undirected for some metrics

# Add directed network specific metrics
in_degree = dict(grn.in_degree())
out_degree = dict(grn.out_degree())

grn_df = pd.DataFrame({
    'gene': list(grn.nodes()),
    'in_degree': [in_degree[node] for node in grn.nodes()],
    'out_degree': [out_degree[node] for node in grn.nodes()],
    'total_degree': [grn_metrics['degree'][node] for node in grn.nodes()],
    'betweenness': [grn_metrics['betweenness'][node] for node in grn.nodes()]
})

# Classify genes by regulatory role
grn_df['role'] = 'Intermediate'
grn_df.loc[grn_df['out_degree'] >= 3, 'role'] = 'Master_Regulator'
grn_df.loc[(grn_df['in_degree'] >= 2) & (grn_df['out_degree'] == 0), 'role'] = 'Target'

print("Gene Regulatory Network Analysis:")
print(grn_df.head())
print(f"\nGene roles:")
print(grn_df['role'].value_counts())

In [ ]:
%%R -i grn_df -w 12 -h 8

library(ggplot2)
library(dplyr)

# Gene regulatory network analysis plot
p5 <- ggplot(grn_df, aes(x = in_degree, y = out_degree, color = role, size = betweenness)) +
    geom_point(alpha = 0.7) +
    scale_color_manual(values = c(
        "Master_Regulator" = "red",
        "Target" = "blue",
        "Intermediate" = "gray"
    )) +
    scale_size_continuous(range = c(2, 6)) +
    labs(
        title = "Gene Regulatory Network Analysis",
        subtitle = "Classification by regulatory role",
        x = "In-degree (Regulated by)",
        y = "Out-degree (Regulates)",
        color = "Gene Role",
        size = "Betweenness\nCentrality"
    ) +
    theme_classic() +
    theme(
        plot.title = element_text(size = 16, hjust = 0.5),
        plot.subtitle = element_text(size = 12, hjust = 0.5),
        axis.title = element_text(size = 12),
        axis.text = element_text(size = 10),
        legend.title = element_text(size = 12),
        legend.text = element_text(size = 10)
    ) +
    annotate("text", x = 0.5, y = 3.5, label = "Master\nRegulators", size = 3, color = "red") +
    annotate("text", x = 2.5, y = 0.2, label = "Targets", size = 3, color = "blue")

print(p5)

## Part 5: Interactive Problem-Solving

### Problem 5.1: Metabolic Network Robustness

Analyze how removing reactions affects network connectivity and flux capacity.

In [ ]:
def analyze_network_robustness(S, reaction_names):
    """
    Analyze metabolic network robustness by removing reactions
    """
    original_rank = linalg.matrix_rank(S)
    original_null_dim = S.shape[1] - original_rank
    
    robustness_results = []
    
    for i, reaction in enumerate(reaction_names):
        # Remove reaction i
        S_reduced = np.delete(S, i, axis=1)
        
        # Calculate new properties
        new_rank = linalg.matrix_rank(S_reduced)
        new_null_dim = S_reduced.shape[1] - new_rank
        
        robustness_results.append({
            'removed_reaction': reaction,
            'original_null_dim': original_null_dim,
            'new_null_dim': new_null_dim,
            'flux_capacity_change': new_null_dim - original_null_dim,
            'is_essential': new_null_dim < original_null_dim
        })
    
    return pd.DataFrame(robustness_results)

# Analyze robustness of our metabolic network
robustness_df = analyze_network_robustness(S_metabolic, reactions)

print("Network Robustness Analysis:")
print(robustness_df)
print(f"\nEssential reactions (reduce flux capacity):")
essential_reactions = robustness_df[robustness_df['is_essential']]
print(essential_reactions[['removed_reaction', 'flux_capacity_change']])

### Problem 5.2: Your Turn - Network Centrality Analysis

**Challenge**: Compare different centrality measures and identify the most "important" proteins in the PPI network.

In [ ]:
# TODO: Implement a function to rank proteins by different centrality measures
def rank_proteins_by_centrality(network_df, top_n=5):
    """
    Rank proteins by different centrality measures
    
    Parameters:
    network_df: DataFrame with centrality measures
    top_n: Number of top proteins to return
    """
    centrality_measures = ['degree', 'betweenness', 'closeness', 'eigenvector']
    
    rankings = {}
    
    for measure in centrality_measures:
        # TODO: Sort proteins by centrality measure and get top_n
        top_proteins = network_df.nlargest(top_n, measure)[['protein', measure]]
        rankings[measure] = top_proteins
    
    return rankings

# Test your implementation
protein_rankings = rank_proteins_by_centrality(network_df)

print("Top 5 proteins by different centrality measures:")
for measure, ranking in protein_rankings.items():
    print(f"\n{measure.capitalize()} centrality:")
    print(ranking.to_string(index=False))

## Summary and Key Takeaways

### What We've Learned

1. **Linear Algebra Applications**: Matrix operations for biological network analysis
2. **Flux Balance Analysis**: Optimization of metabolic networks using linear programming
3. **Principal Component Analysis**: Dimensionality reduction for omics data
4. **Network Theory**: Graph analysis of biological interaction networks
5. **Network Motifs**: Common patterns in biological networks and their functions
6. **Centrality Analysis**: Identifying important nodes in biological networks

### Biological Insights

- **Stoichiometric constraints** limit possible flux distributions in metabolism
- **Hub proteins** are often essential and disease-related
- **Network motifs** represent evolutionary-selected functional modules
- **Principal components** reveal hidden structure in high-dimensional biological data
- **Network robustness** determines system resilience to perturbations

### Mathematical Connections

- **Null space analysis** reveals steady-state solutions
- **Eigenvalue decomposition** underlies both PCA and network centrality
- **Linear programming** optimizes biological objective functions
- **Graph theory** quantifies network topology and function

### Next Steps

- Explore dynamic network models
- Learn about network control theory
- Study multi-layer biological networks
- Apply these methods to real biological datasets

---

**Congratulations!** You've completed the Linear Algebra and Network Theory module of BIO559R Mathematical Foundations. 🎉